# Testing Saved Model from elmo_ner_hid512.ipynb

**Loading necessary libraries**

In [1]:
from keras.models import load_model, model_from_json
import pandas as pd
import numpy as np
import os
import re

**Loading saved model**

In [2]:
# model = load_model('ner_elmo_bilstm.h5')
# model.load_weights('ner_elmo_bilstm_weights.h5')
json_file = open('model2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model2.h5")

**Loading AllenNLP Elmo module to get pre-trained embeddings**

In [3]:
from allennlp.modules.elmo import Elmo,batch_to_ids

In [4]:
datadir = os.path.join('hi', 'elmo')
options_file = os.path.join(datadir, 'hi-d512-options.json')
weight_file = os.path.join(datadir, 'hi-d512-elmo.hdf5')

elmo = Elmo(options_file,weight_file,1,dropout=0)

**Custom sentence to predict tags on -**

In [5]:
# sentence = "उत्तराखंड की राजधानी देहरादून में फरवरी की दुपहरी पिछले दस साल में सबसे गर्म रही".split()
# sentence = "अरे सारथी जम्मू कैसा मौसम है".split()
# sentence = "मुझे बताओ हांगकांग में तापमान क्या है".split()
# sentence =  "मध्य प्रदेश के खरगोन जिले में इस गुरुवार दोपहर तेज बारिश".split()
# sentence = "लक्ष्मण और मेघनाद के मध्य घोर युद्ध हुआ".split()
sentence = "खेत के मध्य मत जाओ".split()

In [6]:
X = [w for w in sentence]
X

['खेत', 'के', 'मध्य', 'मत', 'जाओ']

**Padding the sentences to length = max_len**

In [7]:
max_len = 16

In [8]:
if len(X)<max_len:
    while len(X)<max_len:
        X += ["__PAD__"]
X = [X]

In [9]:
embeds = elmo(batch_to_ids(X))
embedding = embeds["elmo_representations"]

**Converting torch tensor embeddings to embedding matrix (list of lists)**

In [10]:
import torch
import tensorflow as tf

embedding1 = []
for t in embedding:
    np_tensor = t.detach().numpy()
    embedding1 += [np_tensor.tolist()]

embedding = embedding1

**Using model to predict tags on sentences**

In [11]:
p = model.predict(np.array(embedding).reshape(1,16,1024))
p = np.argmax(p, axis=-1)

In [12]:
p

array([[1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]], dtype=int64)

In [13]:
# df = pd.DataFrame(columns=['Sentence #','Word','Tag'])

In [14]:
# data_dir = r'Data/'

In [15]:
# i=1
# data = list()
# with open(data_dir+'nertrnweather.txt',encoding='utf-8') as f:
#     for line in f.readlines():
#         if line=='\n':
#             i+=1
#         else:
#             data=line.split(" ")
#             df=df.append({"Sentence #":f"Sentence {i}","Word":data[0],"Tag":re.sub("\n","",data[1])},ignore_index=True)
# df

In [16]:
# tags = list(set(df["Tag"].values))
# n_tags = len(tags); n_tags
tags = ['B-date',
 'U-location',
 'U-date',
 'L-location',
 'U-weather_type',
 'O',
 'B-location',
 'L-date',
 'L-weather_type',
 'B-weather_type']

In [17]:
p.tolist()

[[1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]

**Printing results from model**

In [18]:
sentence = ""
for w,pred in zip(X[0],p.tolist()[0]):
    if w!= '__PAD__':
        sentence += "{} ({}) ".format(w,tags[pred])
#         print(w,tags[pred])
print(sentence)

खेत (U-location) के (O) मध्य (O) मत (O) जाओ (O) 
